In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

image_path = "/kaggle/input/pestopia/Datasets/Pest_Dataset"
csv_path = "/kaggle/input/pestopia/Datasets/Pesticide_Dataset/Pesticides.csv"

pesticide_df = pd.read_csv(csv_path)

X=[]
y=[]

for label in os.listdir(image_path):
    label_folder= os.path.join(image_path, label)
    for img_file in os.listdir(label_folder):
        img_path=os.path.join(label_folder, img_file)
        img= cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img,(128,128))
            img = img/255.0
            X.append(img)
            y.append(label)

encoder= LabelEncoder()
y_encoded = encoder.fit_transform(y)

X= np.array(X)
y_encoded = np.array(y_encoded)

print("Preprocessing complete")
print("Image shape:", X.shape)
print("Number of classes:", len(np.unique(y_encoded)))

X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

plt.figure(figsize=(12, 6))
plt.hist(y_encoded, bins=len(np.unique(y_encoded)), edgecolor='black')
plt.title("Pest Class Distribution")
plt.xlabel("Encoded Class")
plt.ylabel("Frequency")
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np

tf.config.list_physical_devices('GPU')

y_train_cat= to_categorical(y_train)
y_val_cat= to_categorical(y_val)

model = Sequential([Input(shape= (128,128,3)),
                    Conv2D(32,(3,3), activation='relu'),MaxPooling2D(2,2),
                    Conv2D(64,(3,3), activation='relu'),MaxPooling2D(2,2),
                    Conv2D(128,(3,3), activation='relu'),MaxPooling2D(2,2),
                    Flatten(), Dense(256, activation='relu'),
                    Dropout(0.5), Dense(len(np.unique(y_train)), activation='softmax')])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train_cat, validation_data=(X_val, y_val_cat), epochs=10, batch_size=32)

val_loss, val_acc = model.evaluate(X_val, y_val_cat)
print(f"Validation Accuracy: {val_acc:.2f}")